In [3]:
%pip install google-cloud-aiplatform

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import json
import vertexai
from vertexai.generative_models import GenerativeModel

/Users/petermousses/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
PROJECT='llms-441517'
LOCATION='us-central1'

MODEL_NAME='gemini-1.5-pro'

START_SEQUENCE='Step-by-step solution:\n'
END_SEQUENCE='Final Answer:\n'

PUZZLE_ROOT='./train'
REASONING_ROOT='./reasonings'

In [5]:
SYSTEM_PROMPT="""## View in plaintext
Iterative meta-token generation
### Prompt
Task Overview:
You are annotating a given chain of reasoning with meta-tokens. You will be provided with a description of the meta-tokens and an explanation on how they should be used. Given the previous steps as well as the immediate next step, you will generate and use the meta-tokens to arrive at that next step from the previous ones. You will also be provided the answer to ensure your generation is accurate, but you should not reference it at all. An annotation starts with the meta-token, followed by your usage of the token. Do not conclude meta-tokens with the same one and a slash, only by switching to a different meta-token.

Meta-tokens:
<recall> - This token is used to look over the context and pull out the information that will be immediately relevant to figuring out the next part.
<think> - This token is used to do internal reasoning, separately from normal discussion.
<generate> - The standard mode, used to switch back to normal response. Anything after this token is displayed to the user.

Usage Example:
```
problem statement
...
[relevant info]
...
previous line of reasoning
<recall>[relevant info] to be used in the next conclusion
<think>internal reasoning for the next conclusion
<generate>line of reasoning
```

For instance, if the text was:
```
Clues:
x. Clue

Chain of reasoning:
previous reasoning
next line of reasoning using clue x
```
Then your output would only be:
```
<recall>x. Clue
<think>How you arrive at the next line of reasoning based on the past and the recalled info
<generate>next line of reasoning
```

Problem:
```
Using only the clues below, match the prices to the options from customers and orders. Remember, as with all grid-based logic puzzles, no option in any category will ever be used more than once.

prices : $3.30, $3.40, $3.50, $3.60.
customers : Austin, Charlie, Eduardo, Ira.
orders : cappuccino, eggnog latte, iced americano, mocha.
Clues:
1. The coffee ordered by Austin cost 10 cents less than the eggnog latte.
2. The coffee Charlie ordered cost $3.30.
3. The eggnog latte was either the beverage Charlie ordered or the beverage ordered by Eduardo.
4. Of the coffee that cost $3.60 and the coffee Austin ordered, one was the mocha and the other was the cappuccino.
5. The cappuccino cost somewhat more than the eggnog latte.

Fill the following table to show your final answer.
$3.30 | correct option from customers | correct option from orders
$3.40 | correct option from customers | correct option from orders
$3.50 | correct option from customers | correct option from orders
$3.60 | correct option from customers | correct option from orders
```

Chain of reasoning:
```
<recall>Clue #2: The coffee Charlie ordered cost $3.30.
<think>From Clue #2, we know that Charlie ordered a coffee for $3.30. We need to determine which coffee this is. 
<generate>We can immediately place Charlie and $3.30 in the table.

Next Line of Reasoning: Clue #3: If eggnog latte is either Charlie or Eduardo, and Charlie and Eduardo are in the same category, then eggnog latte does not equal Austin. In short, eggnog latte cannot equal Austin

Final Answer:
$3.30 | Charlie | iced americano
$3.40 | Austin | mocha
$3.50 | Eduardo | eggnog latte
$3.60 | Ira | cappuccino
```

Instructions:
Use the meta-tokens to arrive at the next line of reasoning and ONLY the next line of reasoning. Do not generate further lines of reasoning. Do not abbreviate. There may be steps that use multiple annotations or do not need annotations at all, that is up to you. Ensure each meta-token stays within its own domain; for instance, do not have reasoning in recall annotations, and vice versa.

"""

In [6]:
vertexai.init(
	project=PROJECT, # You can find it by clicking on the project dropdown in the top left.
	location=LOCATION
)

In [7]:
model = GenerativeModel(
	model_name=MODEL_NAME,
	# system_instruction=SYSTEM_PROMPT
)

In [8]:
def get_model_response(prompt):
	response = model.generate_content(prompt, stream=False)
	return response.text

def get_hints(puzzle):
	answer = puzzle['answer']
	return answer[
		(answer.index(START_SEQUENCE) + len(START_SEQUENCE))
		:
		(answer.index(END_SEQUENCE))
	]

def get_answer(puzzle):
	answer = puzzle['answer']
	return answer[
		(answer.index(END_SEQUENCE) + len(END_SEQUENCE))
		:
	]

def get_prompt(question, reasoning_so_far):
	return f"""## View in plaintext
Iterative meta-token generation
### Prompt
Task Overview:
You are annotating a given chain of reasoning with meta-tokens. You will be provided with a description of the meta-tokens and an explanation on how they should be used. Given the previous steps as well as the immediate next step, you will generate and use the meta-tokens to arrive at that next step from the previous ones. You will also be provided the answer to ensure your generation is accurate, but you should not reference it at all. An annotation starts with the meta-token, followed by your usage of the token. Do not conclude meta-tokens with the same one and a slash, only by switching to a different meta-token.

Meta-tokens:
<recall> - This token is used to look over the context and pull out the information that will be immediately relevant to figuring out the next part.
<think> - This token is used to do internal reasoning, separately from normal discussion.
<generate> - The standard mode, used to switch back to normal response. Anything after this token is displayed to the user.

Usage Example:
```
problem statement
...
[relevant info]
...
previous line of reasoning
<recall>[relevant info] to be used in the next conclusion
<think>internal reasoning for the next conclusion
<generate>line of reasoning
```

For instance, if the text was:
```
Clues:
x. Clue

Chain of reasoning:
previous reasoning
next line of reasoning using clue x
```
Then your output would only be:
```
<recall>x. Clue
<think>How you arrive at the next line of reasoning based on the past and the recalled info
<generate>next line of reasoning
```

Problem:
{question}

Chain of reasoning:
{reasoning_so_far}

Instructions:
Use the meta-tokens to arrive at the next line of reasoning and ONLY the next line of reasoning. Do not generate further lines of reasoning. Do not abbreviate. There may be steps that use multiple annotations or do not need annotations at all, that is up to you. Ensure each meta-token stays within its own domain; for instance, do not have reasoning in recall annotations, and vice versa.

"""

def separate_prompt_and_response(prompt, full_response):
	return full_response.replace(prompt, '').strip()

def reasoning_generation_step(question, reasoning_so_far):
	return get_model_response(get_prompt(question, reasoning_so_far))

def is_end_of_reasoning(response):
	return False#response.strip().lower().find('final answer:') > -1

def generate_reasoning_chain(puzzle):
	try:
		response = ''
		# previous_response = puzzle['question']
		while not is_end_of_reasoning(response):
			response += reasoning_generation_step(puzzle['question'], response)
			# print(separate_prompt_and_response(previous_response, response), end='\n----\n') # for debugging
			# previous_response = response
	except KeyboardInterrupt:
		pass # capture keyboard interrupt to save to file later on
	finally:
		return response
	
def generate_answer(puzzle, reasoning):
	return get_model_response(f"""
Generate the final answer for the following puzzle and the reasoning chain. Your response should ONLY include the final answer presented as a table and NOTHING ELSE.

{reasoning}
""")

In [9]:
file_name = '341'
with open(os.path.join(PUZZLE_ROOT, f'{file_name}.json'), 'r') as file:
	os.makedirs(os.path.join(REASONING_ROOT, file_name), exist_ok=True)
	puzzles = json.load(file)
	for sample in puzzles:
		id = sample['id']
		output_file_name = os.path.join(REASONING_ROOT, file_name, f'{id}_reasoning.txt')
		sample_chain = generate_reasoning_chain(sample)
		with open(output_file_name, 'w') as reasoning_file:
			json.dump(sample_chain, reasoning_file)
			print(f'Reasoning {id} saved to {output_file_name}')
		

Reasoning 318 saved to ./reasonings/341/318_reasoning.txt
Reasoning 554 saved to ./reasonings/341/554_reasoning.txt
Reasoning 757 saved to ./reasonings/341/757_reasoning.txt
Reasoning 926 saved to ./reasonings/341/926_reasoning.txt
Reasoning 336 saved to ./reasonings/341/336_reasoning.txt
Reasoning 342 saved to ./reasonings/341/342_reasoning.txt
Reasoning 934 saved to ./reasonings/341/934_reasoning.txt
Reasoning 264 saved to ./reasonings/341/264_reasoning.txt
Reasoning 241 saved to ./reasonings/341/241_reasoning.txt
Reasoning 879 saved to ./reasonings/341/879_reasoning.txt
Reasoning 894 saved to ./reasonings/341/894_reasoning.txt
Reasoning 458 saved to ./reasonings/341/458_reasoning.txt
Reasoning 198 saved to ./reasonings/341/198_reasoning.txt
Reasoning 778 saved to ./reasonings/341/778_reasoning.txt
Reasoning 753 saved to ./reasonings/341/753_reasoning.txt
Reasoning 293 saved to ./reasonings/341/293_reasoning.txt
Reasoning 932 saved to ./reasonings/341/932_reasoning.txt
Reasoning 865 